In [22]:
import torch
import heat as ht
import heat.nn.functional as F
import time

In [23]:
import heat.optim as optim
from heat.utils.data.mnist import MNISTDataset
from heat.utils import vision_transforms

from heat.optim.lr_scheduler import StepLR

In [21]:
class Net(ht.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = ht.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = ht.nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = ht.nn.Dropout2d(0.25)
        self.dropout2 = ht.nn.Dropout2d(0.5)
        self.fc1 = ht.nn.Linear(9216, 128)
        self.fc2 = ht.nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [38]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    t_list = []
    for batch_idx, (data, target) in enumerate(train_loader):
        t = time.perf_counter()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print(
                f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} "
                f"({100.0 * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}"
            )

        t_list.append(time.perf_counter() - t)
    print("average time", sum(t_list) / len(t_list))

In [34]:
transform = ht.utils.vision_transforms.Compose(
        [vision_transforms.ToTensor(), vision_transforms.Normalize((0.1307,), (0.3081,))]
    )

dataset1 = MNISTDataset("./datasets", train=True, transform=transform, ishuffle=False)

In [31]:
train_loader = ht.utils.data.datatools.DataLoader(dataset=dataset1)

In [17]:
device = torch.device("cpu")

In [28]:
model = Net().to(device)

In [19]:
optimizer = optim.Adadelta(model.parameters(), lr=1.0)

dp_optim = ht.optim.DataParallelOptimizer(optimizer, blocking=False)

AttributeError: 'collections.OrderedDict' object has no attribute 'parameters'

In [34]:
scheduler = StepLR(optimizer, step_size=1, gamma=.7)

In [35]:
dp_model = ht.nn.DataParallel(
        model, comm=dataset1.comm, optimizer=dp_optim, blocking_parameter_updates=False
    )

In [42]:
train(model=dp_model, device=device, train_loader=train_loader, optimizer=dp_optim, epoch=1)

/home/jj/anaconda3/envs/wheat/lib/python3.10/site-packages/torch/nn/functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.397120
Train Epoch: 1 [10/60000 (0%)]	Loss: 2.947462
Train Epoch: 1 [20/60000 (0%)]	Loss: 2.470212
Train Epoch: 1 [30/60000 (0%)]	Loss: 2.236562
Train Epoch: 1 [40/60000 (0%)]	Loss: 2.376452
Train Epoch: 1 [50/60000 (0%)]	Loss: 2.188611
Train Epoch: 1 [60/60000 (0%)]	Loss: 2.323920
Train Epoch: 1 [70/60000 (0%)]	Loss: 2.243265
Train Epoch: 1 [80/60000 (0%)]	Loss: 2.938975
Train Epoch: 1 [90/60000 (0%)]	Loss: 2.396703
Train Epoch: 1 [100/60000 (0%)]	Loss: 1.486218
Train Epoch: 1 [110/60000 (0%)]	Loss: 2.278597
Train Epoch: 1 [120/60000 (0%)]	Loss: 2.282209
Train Epoch: 1 [130/60000 (0%)]	Loss: 9.264013
Train Epoch: 1 [140/60000 (0%)]	Loss: 2.285577
Train Epoch: 1 [150/60000 (0%)]	Loss: 1.386107
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.418730
Train Epoch: 1 [170/60000 (0%)]	Loss: 0.961765
Train Epoch: 1 [180/60000 (0%)]	Loss: 2.327511
Train Epoch: 1 [190/60000 (0%)]	Loss: 2.282579
Train Epoch: 1 [200/60000 (0%)]	Loss: 3.266454
Train Epoch: 1 [210/6000

Train Epoch: 1 [1730/60000 (3%)]	Loss: 0.000002
Train Epoch: 1 [1740/60000 (3%)]	Loss: 2.295693
Train Epoch: 1 [1750/60000 (3%)]	Loss: 0.000027
Train Epoch: 1 [1760/60000 (3%)]	Loss: 2.301378
Train Epoch: 1 [1770/60000 (3%)]	Loss: 2.243022
Train Epoch: 1 [1780/60000 (3%)]	Loss: 0.000745
Train Epoch: 1 [1790/60000 (3%)]	Loss: 2.354565
Train Epoch: 1 [1800/60000 (3%)]	Loss: 0.000178
Train Epoch: 1 [1810/60000 (3%)]	Loss: 2.354664
Train Epoch: 1 [1820/60000 (3%)]	Loss: 1.369387
Train Epoch: 1 [1830/60000 (3%)]	Loss: 2.196714
Train Epoch: 1 [1840/60000 (3%)]	Loss: 0.000072
Train Epoch: 1 [1850/60000 (3%)]	Loss: 0.000280
Train Epoch: 1 [1860/60000 (3%)]	Loss: 0.002402
Train Epoch: 1 [1870/60000 (3%)]	Loss: 2.444524
Train Epoch: 1 [1880/60000 (3%)]	Loss: 7.079751
Train Epoch: 1 [1890/60000 (3%)]	Loss: 2.174615
Train Epoch: 1 [1900/60000 (3%)]	Loss: 0.000032
Train Epoch: 1 [1910/60000 (3%)]	Loss: 0.010730
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.335039
Train Epoch: 1 [1930/60000 (3%)]	Loss: 1

Train Epoch: 1 [3440/60000 (6%)]	Loss: 0.000103
Train Epoch: 1 [3450/60000 (6%)]	Loss: 0.000755
Train Epoch: 1 [3460/60000 (6%)]	Loss: 0.115551
Train Epoch: 1 [3470/60000 (6%)]	Loss: 2.384021
Train Epoch: 1 [3480/60000 (6%)]	Loss: 2.279749
Train Epoch: 1 [3490/60000 (6%)]	Loss: 2.309147
Train Epoch: 1 [3500/60000 (6%)]	Loss: 0.002951
Train Epoch: 1 [3510/60000 (6%)]	Loss: 2.309246
Train Epoch: 1 [3520/60000 (6%)]	Loss: 0.003269
Train Epoch: 1 [3530/60000 (6%)]	Loss: 2.353251
Train Epoch: 1 [3540/60000 (6%)]	Loss: 0.001049
Train Epoch: 1 [3550/60000 (6%)]	Loss: 1.026216
Train Epoch: 1 [3560/60000 (6%)]	Loss: 2.385601
Train Epoch: 1 [3570/60000 (6%)]	Loss: 0.000063
Train Epoch: 1 [3580/60000 (6%)]	Loss: 0.000000
Train Epoch: 1 [3590/60000 (6%)]	Loss: 0.000507
Train Epoch: 1 [3600/60000 (6%)]	Loss: 2.343975
Train Epoch: 1 [3610/60000 (6%)]	Loss: 2.259190
Train Epoch: 1 [3620/60000 (6%)]	Loss: 0.000003
Train Epoch: 1 [3630/60000 (6%)]	Loss: 2.339199
Train Epoch: 1 [3640/60000 (6%)]	Loss: 1

Train Epoch: 1 [5150/60000 (9%)]	Loss: 0.000000
Train Epoch: 1 [5160/60000 (9%)]	Loss: 0.022672
Train Epoch: 1 [5170/60000 (9%)]	Loss: 2.283300
Train Epoch: 1 [5180/60000 (9%)]	Loss: 2.103794
Train Epoch: 1 [5190/60000 (9%)]	Loss: 0.000025
Train Epoch: 1 [5200/60000 (9%)]	Loss: 0.000000
Train Epoch: 1 [5210/60000 (9%)]	Loss: 2.427007
Train Epoch: 1 [5220/60000 (9%)]	Loss: 0.000000
Train Epoch: 1 [5230/60000 (9%)]	Loss: 0.000000
Train Epoch: 1 [5240/60000 (9%)]	Loss: 2.470846
Train Epoch: 1 [5250/60000 (9%)]	Loss: 0.000000
Train Epoch: 1 [5260/60000 (9%)]	Loss: 2.260870
Train Epoch: 1 [5270/60000 (9%)]	Loss: 0.002662
Train Epoch: 1 [5280/60000 (9%)]	Loss: 2.091435
Train Epoch: 1 [5290/60000 (9%)]	Loss: 2.266964
Train Epoch: 1 [5300/60000 (9%)]	Loss: 2.086030
Train Epoch: 1 [5310/60000 (9%)]	Loss: 2.294641
Train Epoch: 1 [5320/60000 (9%)]	Loss: 0.000000
Train Epoch: 1 [5330/60000 (9%)]	Loss: 2.472492
Train Epoch: 1 [5340/60000 (9%)]	Loss: 2.463765
Train Epoch: 1 [5350/60000 (9%)]	Loss: 0

...

Train Epoch: 1 [52830/60000 (88%)]	Loss: 2.250451
Train Epoch: 1 [52840/60000 (88%)]	Loss: 2.147132
Train Epoch: 1 [52850/60000 (88%)]	Loss: 2.398451
Train Epoch: 1 [52860/60000 (88%)]	Loss: 0.000000
Train Epoch: 1 [52870/60000 (88%)]	Loss: 0.000010
Train Epoch: 1 [52880/60000 (88%)]	Loss: 2.184048
Train Epoch: 1 [52890/60000 (88%)]	Loss: 0.000023
Train Epoch: 1 [52900/60000 (88%)]	Loss: 2.232321
Train Epoch: 1 [52910/60000 (88%)]	Loss: 2.391439
Train Epoch: 1 [52920/60000 (88%)]	Loss: 0.000000
Train Epoch: 1 [52930/60000 (88%)]	Loss: 0.000000
Train Epoch: 1 [52940/60000 (88%)]	Loss: 0.000000
Train Epoch: 1 [52950/60000 (88%)]	Loss: 2.262546
Train Epoch: 1 [52960/60000 (88%)]	Loss: 2.239823
Train Epoch: 1 [52970/60000 (88%)]	Loss: 0.000000
Train Epoch: 1 [52980/60000 (88%)]	Loss: 0.000000
Train Epoch: 1 [52990/60000 (88%)]	Loss: 2.393402
Train Epoch: 1 [53000/60000 (88%)]	Loss: 2.257103
Train Epoch: 1 [53010/60000 (88%)]	Loss: 2.196795
Train Epoch: 1 [53020/60000 (88%)]	Loss: 0.000000


Train Epoch: 1 [54470/60000 (91%)]	Loss: 0.000000
Train Epoch: 1 [54480/60000 (91%)]	Loss: 2.265489
Train Epoch: 1 [54490/60000 (91%)]	Loss: 0.000020
Train Epoch: 1 [54500/60000 (91%)]	Loss: 2.475289
Train Epoch: 1 [54510/60000 (91%)]	Loss: 0.000000
Train Epoch: 1 [54520/60000 (91%)]	Loss: 0.185941
Train Epoch: 1 [54530/60000 (91%)]	Loss: 2.281036
Train Epoch: 1 [54540/60000 (91%)]	Loss: 2.261041
Train Epoch: 1 [54550/60000 (91%)]	Loss: 2.291150
Train Epoch: 1 [54560/60000 (91%)]	Loss: 0.000000
Train Epoch: 1 [54570/60000 (91%)]	Loss: 2.469464
Train Epoch: 1 [54580/60000 (91%)]	Loss: 21.730728
Train Epoch: 1 [54590/60000 (91%)]	Loss: 2.273241
Train Epoch: 1 [54600/60000 (91%)]	Loss: 0.013936
Train Epoch: 1 [54610/60000 (91%)]	Loss: 0.000116
Train Epoch: 1 [54620/60000 (91%)]	Loss: 0.000516
Train Epoch: 1 [54630/60000 (91%)]	Loss: 0.000000
Train Epoch: 1 [54640/60000 (91%)]	Loss: 0.000073
Train Epoch: 1 [54650/60000 (91%)]	Loss: 0.000000
Train Epoch: 1 [54660/60000 (91%)]	Loss: 0.000000

Train Epoch: 1 [56110/60000 (94%)]	Loss: 2.285809
Train Epoch: 1 [56120/60000 (94%)]	Loss: 0.000000
Train Epoch: 1 [56130/60000 (94%)]	Loss: 2.135054
Train Epoch: 1 [56140/60000 (94%)]	Loss: 0.000000
Train Epoch: 1 [56150/60000 (94%)]	Loss: 2.235330
Train Epoch: 1 [56160/60000 (94%)]	Loss: 2.122796
Train Epoch: 1 [56170/60000 (94%)]	Loss: 0.000000
Train Epoch: 1 [56180/60000 (94%)]	Loss: 2.586998
Train Epoch: 1 [56190/60000 (94%)]	Loss: 2.215444
Train Epoch: 1 [56200/60000 (94%)]	Loss: 2.279108
Train Epoch: 1 [56210/60000 (94%)]	Loss: 2.407500
Train Epoch: 1 [56220/60000 (94%)]	Loss: 0.000000
Train Epoch: 1 [56230/60000 (94%)]	Loss: 2.406346
Train Epoch: 1 [56240/60000 (94%)]	Loss: 2.501212
Train Epoch: 1 [56250/60000 (94%)]	Loss: 2.496922
Train Epoch: 1 [56260/60000 (94%)]	Loss: 0.000000
Train Epoch: 1 [56270/60000 (94%)]	Loss: 2.222702
Train Epoch: 1 [56280/60000 (94%)]	Loss: 0.000000
Train Epoch: 1 [56290/60000 (94%)]	Loss: 0.000000
Train Epoch: 1 [56300/60000 (94%)]	Loss: 0.000285


Train Epoch: 1 [57750/60000 (96%)]	Loss: 0.000369
Train Epoch: 1 [57760/60000 (96%)]	Loss: 0.000001
Train Epoch: 1 [57770/60000 (96%)]	Loss: 2.257670
Train Epoch: 1 [57780/60000 (96%)]	Loss: 0.000000
Train Epoch: 1 [57790/60000 (96%)]	Loss: 2.195679
Train Epoch: 1 [57800/60000 (96%)]	Loss: 2.182559
Train Epoch: 1 [57810/60000 (96%)]	Loss: 2.189631
Train Epoch: 1 [57820/60000 (96%)]	Loss: 2.420958
Train Epoch: 1 [57830/60000 (96%)]	Loss: 2.179585
Train Epoch: 1 [57840/60000 (96%)]	Loss: 2.171293
Train Epoch: 1 [57850/60000 (96%)]	Loss: 2.198920
Train Epoch: 1 [57860/60000 (96%)]	Loss: 0.000000
Train Epoch: 1 [57870/60000 (96%)]	Loss: 2.168226
Train Epoch: 1 [57880/60000 (96%)]	Loss: 1.325068
Train Epoch: 1 [57890/60000 (96%)]	Loss: 0.000000
Train Epoch: 1 [57900/60000 (96%)]	Loss: 0.000199
Train Epoch: 1 [57910/60000 (97%)]	Loss: 2.441449
Train Epoch: 1 [57920/60000 (97%)]	Loss: 0.000000
Train Epoch: 1 [57930/60000 (97%)]	Loss: 2.325280
Train Epoch: 1 [57940/60000 (97%)]	Loss: 0.000149


Train Epoch: 1 [59390/60000 (99%)]	Loss: 2.219559
Train Epoch: 1 [59400/60000 (99%)]	Loss: 2.245457
Train Epoch: 1 [59410/60000 (99%)]	Loss: 2.209817
Train Epoch: 1 [59420/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59430/60000 (99%)]	Loss: 2.289477
Train Epoch: 1 [59440/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59450/60000 (99%)]	Loss: 2.243574
Train Epoch: 1 [59460/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59470/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59480/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59490/60000 (99%)]	Loss: 2.183966
Train Epoch: 1 [59500/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59510/60000 (99%)]	Loss: 2.503404
Train Epoch: 1 [59520/60000 (99%)]	Loss: 2.216824
Train Epoch: 1 [59530/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59540/60000 (99%)]	Loss: 2.616702
Train Epoch: 1 [59550/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59560/60000 (99%)]	Loss: 2.500299
Train Epoch: 1 [59570/60000 (99%)]	Loss: 0.000000
Train Epoch: 1 [59580/60000 (99%)]	Loss: 2.605065


In [43]:
torch.save(model.state_dict(), "mnist_cnn.pt")

In [44]:
! ls 

datasets  eddl.ipynb   heat.ipynb  mnist_cnn.pt  resnet18.onnx	torch.model
docs	  eddlShit.py  mlruns	   model.pickle  tmp		vgg16.onnx


In [8]:
with open('mnist_cnn.pt', 'rb') as f:
    model = torch.load(f)

In [10]:
#mlflow.pytorch.autolog()

In [11]:
## simpler case

In [60]:
class SimpleNet(ht.nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28*28, 10)
        
    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))
    
    def trainig_step(self, batch, batch_nb):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        pred = logits.argmax(dim=1)
        
        self.log('train_loss', loss, on_epoch=True)
        
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=.01)
    
    

In [24]:
dataset1 = MNISTDataset("./datasets", train=True, ishuffle=False)

In [47]:
simple_model = SimpleNet().to(device)

In [48]:

dp_optim = ht.optim.DataParallelOptimizer(simple_model.configure_optimizers(), blocking=False)

In [49]:
dp_model = ht.nn.DataParallel(
        simple_model, comm=dataset1.comm, optimizer=dp_optim, 
    blocking_parameter_updates=False
    )

In [35]:
train_loader = ht.utils.data.datatools.DataLoader(dataset=dataset1)

In [64]:
def small_train(model, device, train_loader, optimizer, epoch):
    model.train()
    t_list = []
    for batch_idx, (data, target) in enumerate(train_loader):
        t = time.perf_counter()
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        mlflow.log_metric('train_loss', value=loss)
        
       

In [31]:
import mlflow
import mlflow.pytorch

In [ ]:
# only works with lightining
#mlflow.pytorch.autolog()

In [66]:
with mlflow.start_run() as run:
    for i, p in enumerate(simple_model.parameters()):
        mlflow.log_param(key=f"param{i}", value=p.size())
        
    small_train(model=dp_model, device=device, train_loader=train_loader, optimizer=dp_optim, epoch=1)
    
    mlflow.pytorch.log_model(pytorch_model=simple_model, artifact_path='path')

In [51]:
mlflow.end_run()

/home/jj/anaconda3/envs/wheat/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [41]:
# mlflow.pytorch.load_model